In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# !pip install gtts pydub audioop-lts
from pydub import AudioSegment
import time
from datetime import date
import os
import copy
import pandas as pd
from gtts import gTTS
from collections import defaultdict
from utils_data import (
    load_raw_data, check_dups, filter_df_to_vocab_of_interest, 
    fill_default_settings
    )
from utils_audio import (
    create_tts_files_for_one_vocab_word, compute_pinyin_and_create_recordings,
    combine_audio_files_and_compute_durations
)
from video_file_configs.a002_second_refresh import dict_recordings
pd.options.display.max_columns = 100
pd.options.display.max_rows = 130


# 0. Define audio

In [4]:
audio_text_lists = [
# Introduction
    [
    """
知识点汇总
1.根据题型及分数安排复习重点
在《艺术概论》考卷上,其题型主要有择题、问答题、作品赏析题、论述题等,
每个题型所占的分值比例各不相同,出题思路也有所差异,因此,考生可以将不同
题型考察的重点对照考试大纲进行标注,然后根据标注的内容认真学习,将重点知
识在脑海中都过一遍,并搭配一定的真题练习进行巩固。
""",

    """
2.根据试题内容选择备考重点
在备考《艺术概论》的选择题时,我们不难发现,无论真题的题型如何变更,
其考察的内容仍然是基础知识。虽然出题方式不同,但考查的知识点难度大同小异。
如考查作品、作家等知识点,这都是各章节中的原题。考生可以用特殊颜色的记号
笔将这些在题中出现的知识点进行标注,然后按照标注有选择的进行背诵,这不仅
降低了我们的备考难度,也能使我们的复习更具备针对性。 
""",],

# Chapter 1
    [
    """
第一章——绪论
【要点】了解本学科的研究对象、学科性质、学科发展及任务、研究方
法等。
    """,

    """
(一)研究对象及学科性质
1.研究对象
艺术概论的研究对象是人类的艺术活动,以及与之相关的概念、范畴、理论和方法等。
艺术有广义与狭义之分。
※广义的艺术包括实用艺术(建筑、园林、工艺美术与现代设计)、造型艺术(绘画、
雕塑、摄影、书法等)、表情艺术(音乐、舞蹈等)、综合艺术(戏剧、戏曲、电影、电视艺
术等)、语言艺术(诗歌、散文、小说等)。
※狭义的艺术专指语言艺术以外的其他艺术种类。语言艺术也称文学,人们有时把文学
与狭义的艺术并称为文艺。
我们这门“艺术概论”课程所使用的是广义的艺术概念。艺术活动是指人类从事的一切
艺术行为。
    """,

    """
2.学科性质
艺术概论是一门研究艺术活动基本规律的学科,是阐述艺术的基本性质、艺术创作与传
播接受系统、艺术种类和特点的科学体系。
人类的艺术活动是伴随着人类社会的产生而产生的。早在数万年前的原始社会,人类就
开始了音乐、舞蹈、绘画等艺术活动。随着社会的不断发展和进步,人类的艺术活动越来越
丰富,随之产生了一系列艺术思想与艺术理论。如西方古希腊的许多著名哲学家、思想家、
艺术家在他们的著作中提出了许多艺术思想,中国自古以来产生的大量的文论、诗论、乐论、
画论等也都是对艺术的研究。这些都为艺术学的产生奠定了基础。
与艺术学联系密切的是美学,于 18 世纪中叶发展成为独立学科的。认为艺术即美,美
学即艺术哲学。
    """,

    """
19 世纪末,德国的康拉德·费德勒(1841-1895)首先提岀了“美学的根本
问题与艺术哲学的根本问题完全不同”的新见解,提岀“美=艺术”的公式不能成立,把美
与艺术、美学和艺术问题的研究区别开来,被称为“艺术学之父”。继费德勒从对象角度提
出艺术学的独立性之后,德国的格罗赛(1862-1927)又着重从方法论上建立艺术科学。他的
《艺术的起源》是艺术社会学的重要著作之一。
进入 20 世纪之后,德国的狄索瓦(1867-1947)、乌提兹(1883-1965)等大力倡导一般
艺术学,出版了一系列学术专著,产生了广泛影响,使艺术学在德国及其他一些国家确立了
学科地位。20 世纪二三十年代,日本、苏联等国相继开展了对艺术学的研究和探讨,我国
也出现了一些艺术学方面的译著和著作,标志着艺术学研究的进一步广泛和深入。
    """,

    """
    (二)学科任务
1.系统地阐述艺术的本质及其特性,确立科学的、进步的艺术观。
艺术观是指对艺术本质、艺术的一般规律及相关原理的观点,艺术观是世界观不可或缺
的组成部分,正确的艺术观需要在先进的、科学的世界观基础上建立。马克思主义的辩证唯
物主义和历史唯物主义世界观是指导我们认识和分析艺术现象的基本岀发点,用马克思主义
基本理论和观点分析艺术现象,总结概括艺术规律,确立科学的、进步的艺术观是艺术概论
学科的基本任务之一。
    """,

    """
2.深入论述艺术与社会历史发展的关系,坚持文艺“以人民为中心”的根本宗旨。
马克思主义认为,人民对美好生活的向往是社会发展的根本动力,艺术活动的源泉是人
民的社会实践,艺术源于生活,人民的社会实践活动是艺术创作取之不尽、用之不竭的源泉。
艺术创作活动要坚持以人民为中心,坚持为人民创作;只有深入实践、深入群众、深入生活、
努力创作出人民群众喜爱的精品力作,才能实现艺术家的价值,才能实现艺术服务社会发展
的价值。
    """,

    """
3.了解艺术活动系统各个环节之间的密切联系,探讨艺术活动规律与特点。
艺术活动是由不同环节和部分构成的,这些环节和部分之间存在着有机联系,艺术理论
研究的任务之一就是通过对艺术现象的分析,科学地认识艺术活动系统的各个组成部分和环
节,辨析艺术系统内部的构成规律和艺术活动的内在规律与特点。
    """,

    """
4.指导人们遵循审美规律和艺术规律进行能动的创作、传播、接受和批评。
艺术概论揭示艺术创作规律和审美接受规律,这些规律来源于艺术实践,又反过来指导
人们的艺术实践,使人们自觉地按照艺术规律进行艺术创作、艺术批评和艺术鉴赏。
    """,

    """
(三)研究方法
1.坚持辩证唯物主义和历史唯物主义的基本原则。
2.采用逻辑的和其他行之有效的科学方法,研究艺术活动的实践与发展。
3.运用艺术学科特有的研究方法,研究艺术领域的各种问题。

绪论部分对艺术概论进行简要介绍,通过这部分的学习,应主要了解艺术学是一门什么
样的学科,根据考试大纲,这部分内容所占分数比例为 1%左右。 第二章——艺术活动
    """,
    ],

# Chapter 2
    [
    """
    """,

    """
    """,

    """
    """,

    """
    """,

    """
    """,

    """
    """,

    """
    """,
    ],

# Chapter x
    [
    """
    """,

    """
    """,

    """
    """,

    """
    """,

    """
    """,

    """
    """,

    """
    """,
    ],
]

In [5]:
audio_final_path = 'audio_files/products/aji/20250923/'
if not os.path.exists(audio_final_path):
    os.mkdir(audio_final_path)
print(audio_final_path)

audio_files/products/aji/20250923/


# 2. Compute and save recordings

In [11]:
entire_start_time = time.time()
for chapter_i, chapter_audio_list in enumerate(audio_text_lists):
    for chapter_audio_i, audio_text in enumerate(chapter_audio_list):
        audio_file_name = f"{audio_final_path}{chapter_i}_{chapter_audio_i}.mp3"
        if not os.path.exists(audio_file_name):
            current_start_time = time.time()
            try:
                gTTS(audio_text, lang='zh').save(audio_file_name)
            except:
                print(f"!!!!!!! FAILURE, wait 52 seconds, {chapter_i}, {chapter_audio_i} !!!!!!!\n{audio_text}")
                time.sleep(52)
                try:
                    gTTS(audio_text, lang='zh').save(audio_file_name)
                except:
                    print(f"!!!!!!! FAILURE, wait 258 seconds, {chapter_i}, {chapter_audio_i} !!!!!!!\n{audio_text}")
                    time.sleep(258)
                    gTTS(audio_text, lang='zh').save(audio_file_name)
            print(f"{(time.time()-entire_start_time):5.1f}s, {(time.time()-current_start_time):5.2f}s, {chapter_i}, {chapter_audio_i}")
        else:
            print(f"{(time.time()-entire_start_time):5.1f}s, ALREADY COMPUTED, {chapter_i}, {chapter_audio_i}")
    if chapter_i > 0:
        break

  0.0s, ALREADY COMPUTED, 0, 0
  0.0s, ALREADY COMPUTED, 0, 1
  0.0s, ALREADY COMPUTED, 1, 0
  0.0s, ALREADY COMPUTED, 1, 1
  0.0s, ALREADY COMPUTED, 1, 2
  0.0s, ALREADY COMPUTED, 1, 3
 23.1s, 23.10s, 1, 4
 41.6s, 18.54s, 1, 5
 49.1s,  7.50s, 1, 6
 59.9s, 10.81s, 1, 7
 77.1s, 17.19s, 1, 8


# 3. Make whole recordings per chapter

In [ ]:
entire_start_time = time.time()
for chapter_i, chapter_audio_list in enumerate(audio_text_lists):
    # Construct list of individual audio files
    audio_files_this_chapter = []
    for chapter_audio_i, audio_text in enumerate(chapter_audio_list):
        audio_file_name = f"{audio_final_path}{chapter_i}_{chapter_audio_i}.mp3"
        audio_files_this_chapter.append(AudioSegment.from_mp3(audio_file_name))

    # Concat audio files for this chapter
    concat_file_name = f"{audio_final_path}艺术概论资料汇总_chapter{chapter_i}.mp3"
    audio_concat = audio_files_this_chapter[0]
    for audio in audio_files_this_chapter[1:]:
        audio_concat += audio

    # Export audio
    audio_concat.export(concat_file_name, format="mp3")
    print(f"{(time.time()-entire_start_time):.1f}s, {concat_file_name}")

    if chapter_i > 0:
        break

0.6s, audio_files/products/aji/20250923/concat_0.mp3
2.8s, audio_files/products/aji/20250923/concat_1.mp3
